In [2]:
# Import dependencies

import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt


In [12]:
# Import .CSV files

df_la2019 = pd.read_csv(r"data/Los Angeles 2019 data.csv")
df_la2020 = pd.read_csv(r"data/Los Angeles 2020 data.csv")
df_la2021 = pd.read_csv(r"data/Los Angeles 2021 data.csv")


In [19]:
df_la2019_parsed = df_la2019[['Date', 'CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_la2019_parsed['Date'] = pd.to_datetime(df_la2019_parsed['Date'])
df_la2019_parsed.set_index('Date', inplace=True)
df_la2019_parsed.dropna(inplace=True)
df_la2019_cleaned = df_la2019_parsed[df_la2019_parsed['Local Site Name'] == 'Compton']
df_la2019_cleaned


,CBSA Name,Local Site Name,Daily AQI Value,Daily Mean PM2.5 Concentration
Date,,,,
2019-01-03,"Los Angeles-Long Beach-Anaheim, CA",Compton,57,12.1
2019-01-09,"Los Angeles-Long Beach-Anaheim, CA",Compton,77,23.0
2019-01-12,"Los Angeles-Long Beach-Anaheim, CA",Compton,55,11.1
2019-01-15,"Los Angeles-Long Beach-Anaheim, CA",Compton,41,7.3
2019-01-18,"Los Angeles-Long Beach-Anaheim, CA",Compton,56,11.9
...,...,...,...,...
2019-12-27,"Los Angeles-Long Beach-Anaheim, CA",Compton,62,15.1
2019-12-28,"Los Angeles-Long Beach-Anaheim, CA",Compton,68,18.1
2019-12-29,"Los Angeles-Long Beach-Anaheim, CA",Compton,75,22.2
